# 01_Preprocessing

Goal: Prepare data to be used in model training.
1. Combining datasets into one homogenous human-readable dataset, ensuring no empty cells, with the text **Article** and boolean **Truth** columns.
2. Remove any punctuation and error-prone text.
3. Tokenize the data for NLP.

Output:
1. Human-readable dataset
2. Reduced dataset
3. Tokenized dataset